In [130]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [131]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")
print(data_initial.shape)

(92904, 806)


In [132]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.6, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, train_size = 0.5, test_size=0.5, random_state=45)

# Above two lines in unison accomplish a 60-20-20 split for train-validation-test

In [133]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 1000
val_batch_size = 1000
test_batch_size = 1000

cuda
cuda


In [134]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [135]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.batch_norm = nn.BatchNorm1d(self.hidden_size)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc5 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc6 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm(x)

        x = self.fc6(x)
        
        return x


In [127]:
import optuna

num_epochs = 20
trial_count = 30

learning_rate = 0.01
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += train_batch_size
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        #model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += val_batch_size
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 300, 600)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    print("learning rate:", lr)
    print("weight_decay:", weight_decay)
    print("hidden_size:", hidden_size)
    print("dropout_rate:", dropout_rate)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

study = optuna.create_study(direction="minimize")
input_dim = 529
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-04-24 08:49:47,669] A new study created in memory with name: no-name-1022a00e-38a9-4885-b16f-da43eaf3e707


learning rate: 0.00012642948658038954
weight_decay: 0.00026927668924401894
hidden_size: 506
dropout_rate: 0.3299413377976169

EPOCH 1 of 20

Training loss: 0.00023099914991429874
Validation loss: 1.488502916732901e-05

EPOCH 2 of 20

Training loss: 0.00022270005755126477
Validation loss: 1.477311683916732e-05

EPOCH 3 of 20

Training loss: 0.00022056486723678453
Validation loss: 1.5022605461509604e-05

EPOCH 4 of 20

Training loss: 0.00021877820257629668
Validation loss: 1.4589844075472732e-05

EPOCH 5 of 20

Training loss: 0.00022054661278213773
Validation loss: 1.5205525450016323e-05

EPOCH 6 of 20

Training loss: 0.00022020393769655908
Validation loss: 1.513423001099574e-05

EPOCH 7 of 20

Training loss: 0.00021840163000992366
Validation loss: 1.5421648656851367e-05

EPOCH 8 of 20

Training loss: 0.00022046152794999736
Validation loss: 1.4000161109786285e-05

EPOCH 9 of 20

Training loss: 0.00021731837865497384
Validation loss: 1.4305509381780499e-05

EPOCH 10 of 20

Training loss: 

[I 2023-04-24 09:03:55,669] Trial 0 finished with value: 1.3796966424898097e-05 and parameters: {'lr': 0.00012642948658038954, 'weight_decay': 0.00026927668924401894, 'hidden_size': 506, 'dropout_rate': 0.3299413377976169}. Best is trial 0 with value: 1.3796966424898097e-05.


Validation loss: 1.3796966424898097e-05
learning rate: 0.000699683498535688
weight_decay: 0.0007769644919088114
hidden_size: 427
dropout_rate: 0.27449538624481873

EPOCH 1 of 20

Training loss: 0.00015142153735671725
Validation loss: 2.724551880045941e-05

EPOCH 2 of 20

Training loss: 0.00013612704378153598
Validation loss: 2.829885355343944e-05

EPOCH 3 of 20

Training loss: 0.00013613261242530176
Validation loss: 2.78576890888967e-05

EPOCH 4 of 20

Training loss: 0.0001356426484084555
Validation loss: 2.855762564822247e-05

EPOCH 5 of 20

Training loss: 0.00013632835195000683
Validation loss: 2.8344984509442982e-05

EPOCH 6 of 20

Training loss: 0.00013609965717686074
Validation loss: 2.739282687635798e-05

EPOCH 7 of 20

Training loss: 0.00013672318083367177
Validation loss: 2.799677191988418e-05

EPOCH 8 of 20

Training loss: 0.00013876458310655186
Validation loss: 2.776258997619152e-05

EPOCH 9 of 20

Training loss: 0.00013568293715694122
Validation loss: 2.6440147898699107e-05


[I 2023-04-24 09:18:05,846] Trial 1 finished with value: 2.7079321051898756e-05 and parameters: {'lr': 0.000699683498535688, 'weight_decay': 0.0007769644919088114, 'hidden_size': 427, 'dropout_rate': 0.27449538624481873}. Best is trial 0 with value: 1.3796966424898097e-05.


...

Validation loss: 1.3212195205453195e-05
learning rate: 0.0005309307973717332
weight_decay: 0.00014742404024621934
hidden_size: 484
dropout_rate: 0.3860689783841833

EPOCH 1 of 20

Training loss: 0.00018074161718998637
Validation loss: 4.751103636073439e-06

EPOCH 2 of 20

Training loss: 0.00016704018307583672
Validation loss: 5.030188292853142e-06

EPOCH 3 of 20

Training loss: 0.00016902467528624195
Validation loss: 4.9435848639787815e-06

EPOCH 4 of 20

Training loss: 0.00016581380580152783
Validation loss: 4.913289087677472e-06

EPOCH 5 of 20

Training loss: 0.00016926879595432963
Validation loss: 4.893624395328133e-06

EPOCH 6 of 20

Training loss: 0.00016803960848067487
Validation loss: 4.847248293165313e-06

EPOCH 7 of 20

Training loss: 0.00016508974800152437
Validation loss: 4.9960451494706304e-06

EPOCH 8 of 20

Training loss: 0.00016895097254642419
Validation loss: 4.998556436284592e-06

EPOCH 9 of 20

Training loss: 0.00016532963940075465
Validation loss: 4.801365117983598e-

[I 2023-04-24 15:40:29,232] Trial 28 finished with value: 4.975564814613838e-06 and parameters: {'lr': 0.0005309307973717332, 'weight_decay': 0.00014742404024621934, 'hidden_size': 484, 'dropout_rate': 0.3860689783841833}. Best is trial 12 with value: 2.1860215040904127e-06.


Validation loss: 4.975564814613838e-06
learning rate: 0.010787734228201666
weight_decay: 0.00021604091245710662
hidden_size: 598
dropout_rate: 0.42819583184423016

EPOCH 1 of 20

Training loss: 0.0003817998929215329
Validation loss: 5.653674194687291e-05

EPOCH 2 of 20

Training loss: 0.00015136216820350716
Validation loss: 5.7927238117707406e-05

EPOCH 3 of 20

Training loss: 0.0001521312723468457
Validation loss: 5.353460696182753e-05

EPOCH 4 of 20

Training loss: 0.00015309096460363694
Validation loss: 5.4432138212417304e-05

EPOCH 5 of 20

Training loss: 0.00016049132575946196
Validation loss: 6.000349611828202e-05

EPOCH 6 of 20

Training loss: 0.00015396126226655075
Validation loss: 5.7566349835772266e-05

EPOCH 7 of 20

Training loss: 0.00015397739250745093
Validation loss: 6.005897235713507e-05

EPOCH 8 of 20

Training loss: 0.00015581066933061395
Validation loss: 5.652409007674769e-05

EPOCH 9 of 20

Training loss: 0.00015118630842438766
Validation loss: 5.7467999230874215e-0

[I 2023-04-24 15:54:40,593] Trial 29 finished with value: 5.827802203987774e-05 and parameters: {'lr': 0.010787734228201666, 'weight_decay': 0.00021604091245710662, 'hidden_size': 598, 'dropout_rate': 0.42819583184423016}. Best is trial 12 with value: 2.1860215040904127e-06.


Validation loss: 5.827802203987774e-05
{'lr': 0.008527084096275877, 'weight_decay': 6.071847158024609e-06, 'hidden_size': 352, 'dropout_rate': 0.4894615406787279}


In [128]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

num_epochs = 100 # training for more epochs on best model (which will be saved)

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params2.pt")
# save the trained model
torch.save(model, "faceoffs_best_model2.pt")

{'lr': 0.008527084096275877, 'weight_decay': 6.071847158024609e-06, 'hidden_size': 352, 'dropout_rate': 0.4894615406787279}
Epoch 1 Training loss: 0.28972535671865834
Epoch 2 Training loss: 0.004657728079889368
Epoch 3 Training loss: 0.0015278962257850384
Epoch 4 Training loss: 0.0015325829195457377
Epoch 5 Training loss: 0.0015244994740766873
Epoch 6 Training loss: 0.0015299063026239829
Epoch 7 Training loss: 0.0015299553155533171
Epoch 8 Training loss: 0.0015306515415431932
Epoch 9 Training loss: 0.0015561322434223257
Epoch 10 Training loss: 0.001542125947058334
Epoch 11 Training loss: 0.0015130238004660765
Epoch 12 Training loss: 0.0015042372594637396
Epoch 13 Training loss: 0.0014984798368199595
Epoch 14 Training loss: 0.001497330830066598
Epoch 15 Training loss: 0.0014981602767615446
Epoch 16 Training loss: 0.00149605860081335
Epoch 17 Training loss: 0.0014988877415557259
Epoch 18 Training loss: 0.0014982278184366546
Epoch 19 Training loss: 0.001497928219448243
Epoch 20 Training l

In [129]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 0.0015
True labels: [0.00920247, 0.0160865, 0.00920247, 0.0109509, 0.0, 0.0, 0.0115287, 0.00920247, 0.0, 0.029275, 0.0100969, 0.0, 0.0166409, 0.00927501, 0.00920247, 0.0, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.00920247, 0.013338, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.150549, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.0, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.0194755, 0.0860846, 0.0287319, 0.0107397, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.0, 0.0188868, 0.00920247, 0.0856539, 0.00920247, 0.00920247, 0.192434, 0.00920247, 0.0405562, 0.00920247, 0.00925497, 0.00920247, 0.00920247, 0.0121461, 0.0676457, 0.0390529, 0.00920247, 0.00920247, 0.0941837, 0.00920247, 0.0376101, 0.0584831, 0.0751837, 0.00920247, 0.0457924, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0600364, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.026531, 0.00920247, 0.00920247, 0.0611948, 0.00920247, 0.00920247, 0.00920247, 0.0,

In [139]:
num_epochs = 15

learning_rate = 0.01
loss_func = nn.MSELoss()
input_dim = 799
best_params = {'lr': 0.008527084096275877, 'weight_decay': 6.071847158024609e-06, 'hidden_size': 352, 'dropout_rate': 0.4894615406787279}
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"]).to(device)
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params_revisited.pt")
# save the trained model
torch.save(model, "faceoffs_best_model_revisited.pt")

Epoch 1 Training loss: 0.1506440787010693
Epoch 2 Training loss: 0.0016994934973938922
Epoch 3 Training loss: 0.001523042900121904
Epoch 4 Training loss: 0.0015213169535854831
Epoch 5 Training loss: 0.0015361550987498568
Epoch 6 Training loss: 0.0015812035562703386
Epoch 7 Training loss: 0.0015455518088336767
Epoch 8 Training loss: 0.0015551871349868765
Epoch 9 Training loss: 0.0015569875457523657
Epoch 10 Training loss: 0.0015713759369516214
Epoch 11 Training loss: 0.0015378109237644821
Epoch 12 Training loss: 0.0014997269401127206
Epoch 13 Training loss: 0.0014953390075658848
Epoch 14 Training loss: 0.0014946150331525132
Epoch 15 Training loss: 0.0014957960707501375


In [141]:
test_loss = 0
y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x799 and 529x352)